# K-Nearest Neighbors

The following code trains a K-Nearest Neighbors Classifier to predict the distance between the Raspberrry Pis. The code can be run repeatedly with varying parameter grids to narrow in on the optimal parameters.

In [ ]:
# Config
from pi_pact_sort import categorize, bin_categorize

FEATURES: list = ['RSSI', 'HUMIDITY', 'PRESSURE'] # Contains strings 'RSSI', 'HUDMITIY', and/or 'PRESSURE'
CATEGORIZE_FUNC = bin_categorize # One of the two binning functions in pi_pact_sort.py

# Automatically configure other variables
if len(FEATURES) > 1:
    if len(FEATURES) == 2:
        if 'HUMIDITY' in FEATURES:
            feature_str = '3varH'
        else:
            feature_str = '3varP'
    else:
        feature_str = '4var'
else:
    feature_str = '2var'

if CATEGORIZE_FUNC == categorize:
    label_str = '3b'
else:
    label_str = 'binary'

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

DROP_COLUMNS = ['ADDRESS', 'TIMESTAMP', 'UUID', 'MAJOR', 'MINOR', 'TX POWER', 'TEMPERATURE',  
                'PITCH', 'ROLL', 'YAW', 'SCAN']
for feature in ['RSSI', 'HUMIDITY', 'PRESSURE']:
    if feature not in FEATURES:
        DROP_COLUMNS.append(feature)
SAMPLE_SIZE = 30000
np.random.seed(0)


"""Trains a K-Nearest Neighbors classifier to predict a distance range given RSSI values and other variables.
"""

# Initialize DataFrame
data: pd.DataFrame = pd.DataFrame(columns=['DISTANCE', ] + FEATURES)
data_copy: pd.DataFrame = data.copy()
csv_file: Path
for csv_file in Path('.').glob('indoor-noObstruct-SenseHat*/*.csv'):
    datapart: pd.DataFrame = pd.read_csv(csv_file)
    for column in DROP_COLUMNS:
        if column in datapart.columns:
            datapart = datapart.drop([column], 1)
    data_copy = data_copy.append(datapart)

# Categorize distance
data_copy['DISTANCE'] = data_copy['DISTANCE'].map(CATEGORIZE_FUNC)

# Sample data from each distance category
for value in data_copy['DISTANCE'].unique():
    datapart = data_copy[data_copy.DISTANCE == value]
    datapart = datapart.sample(SAMPLE_SIZE, random_state=1)
    data = data.append(datapart)

# Assign features and labels
X: np.array = data.drop(['DISTANCE'], 1).to_numpy(dtype='float32')
y: np.array = data['DISTANCE'].to_numpy(dtype=int)

# Preprocessing
steps = [('min_max', MinMaxScaler()),
         ('interactions', PolynomialFeatures(interaction_only=True,
                                             include_bias=False))]
if len(FEATURES) > 1:
    steps.append(('dim_reduce', PCA()))
preprocessing = Pipeline(steps=steps)
pipe = Pipeline(steps=[('preprocessing', preprocessing),
                       ('model', KNeighborsClassifier(metric='minkowski'))])

In [ ]:
# Hyperparameter tuning
import math

start = 0
stop = 500
step = 50
n_neighbors: list = list(2 * np.arange(start, stop, step) + 1)
param_grid = {'model__n_neighbors': n_neighbors, 'model__p': [1, 2, 3, 4]}
if len(FEATURES) > 1:
    param_grid.update({'preprocessing__dim_reduce__n_components': [i for i in range(2, 2**len(FEATURES)-1)]})

while math.gcd(step, 10) > 1:
    grid = GridSearchCV(pipe, param_grid, n_jobs=2)
    grid.fit(X, y)
    best_n_neighbors = grid.best_params_['model__n_neighbors']
    if best_n_neighbors != n_neighbors[0]:
        start = ((best_n_neighbors - 2*step) - 1) // 2
    
    if best_n_neighbors != n_neighbors[-1]:
        stop = ((best_n_neighbors + 2*step) - 1) // 2
    
    step = step // math.gcd(step, 10) 
    n_neighbors = list(2 * np.arange(start, stop, step) + 1)
grid = GridSearchCV(pipe, param_grid, n_jobs=2)
grid.fit(X, y)

In [ ]:
# Print optimal parameters and accuracy
print(grid.best_params_)
print('accuracy =', grid.best_score_)

In [ ]:
# Pickle model
import pickle
with open(f"knn-models/{feature_str}-{label_str}-polyFeatures-dimReduce-knn-model.pickle", "wb") as f:
    pickle.dump(grid.best_estimator_, f)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve

# Plot an ROC curve
if CATEGORIZE_FUNC == bin_categorize:
    probs = grid.best_estimator_.predict_proba(X)
    fpr, tpr, _ = roc_curve(y, probs[:, 1])
    plt.plot(fpr, tpr)
    plt.title('ROC curve')
    plt.xlabel('false positive rate')
    plt.ylabel('true positive rate')
    plt.xlim(0,)
    plt.ylim(0,)
    plt.savefig(str(Path(f'knn-models/{feature_str}-{label_str}-polyFeatures-dimReduce-knn-model-roc-curve.png')))